In [1]:
#!/usr/bin/python3
import math
import json


states = [1,2,3,4,5]
utterances = [1,2,3,4,5]
quds = [ "state", "valence", "arousal"]
valences = ["positive", "negative"]
arousals = ["high", "low"]
affects = [(v, a) for v in valences for a in arousals]
contexts = ["WC1", "WC2","WC3","WC4","WC5","WC6","WC7","WC8","WC9"]

# For each context, prior over the states
prior_states = eval(open('irony/prior_states.json', 'r').read())
print("prior states: ", prior_states)
# For each state, prior over valence+arousal combination
prior_affect = eval(open('irony/prior_affect.json', 'r').read())

# Prior over QUDs
# hmmm was this ever calculated in the paper?
prior_quds = {
    "state":    0.3,
    "valence":  0.3,
    "arousal":  0.4,
}

rationality_factor = 1.0

# the q function in the paper
# 's' is the state of the world, 'A' reps the speaker's affect toward the state
# this serves as a projection from full meaning spact to subset of speaker's interest
def qud(q, s, A):
    if q == "state": return s
    if q == "valence": return A[0]
    if q == "arousal": return A[1]
    print("error")


def literal_listener(s, A, u):
    if s != u:          #if state does not == utterance. This is the basic RSA that returns priors for true states.
        return 0.0
    else:
        return prior_affect[s][A] #so the literal listener will return literally whatever the qud asks for. but what happens if the qud is the state? should we return the probability of a weather state happening? If it doesn't it probs won't change behavior because they're all uniform.


# the U function in the paper, without the log
# this models info gained by listener about topic of interest
# where q is the intended QUD

def exp_utility(u, s, A, q):
    sum = 0.0
    for sp in states:
        for Ap in affects:
            if qud(q, s, A) == qud(q, sp, Ap):
                sum += literal_listener(sp, Ap, u)
    return sum

# the S function in the paper, normalized
# "the speaker S chooses an utterance according to a softmax decision rule"
# in the paper, it says the base is e. How come it's the the utility ftn^ rationality factior here?
def speaker(u, s, A, q):
    norm = 0.0
    for up in utterances:
        norm += math.pow(exp_utility(up, s, A, q), rationality_factor)
    return math.pow(exp_utility(u, s, A, q), rationality_factor) / norm

# the pragmatic L function in the paper, unnormalized
# this is the ftn that models the ambiguity behind QUD, particularly it's held in the sum
def unnorm_pragmatic_listener(s, A, u, context):
    sum = 0.0
    for q in quds:
        sum += prior_quds[q] * speaker(u, s, A, q)
    return prior_states[context][s] * prior_affect[s][A] * sum


# the pragmatic L function in the paper, normalized
def pragmatic_listener(s, A, u, context):
    norm = 0.0
    for sp in states:
        for Ap in affects:
            norm += unnorm_pragmatic_listener(sp, Ap, u, context)
    return unnorm_pragmatic_listener(s, A, u, context) / norm



prior states:  {'WC5': {1: 0.018518518518518517, 2: 0.018518518518518517, 3: 0.18518518518518517, 4: 0.5925925925925926, 5: 0.18518518518518517}, 'WC7': {1: 0.09259259259259259, 2: 0.4074074074074074, 3: 0.3333333333333333, 4: 0.14814814814814814, 5: 0.018518518518518517}, 'WC2': {1: 0.018518518518518517, 2: 0.018518518518518517, 3: 0.018518518518518517, 4: 0.05555555555555555, 5: 0.8888888888888888}, 'WC9': {1: 0.5555555555555556, 2: 0.35185185185185186, 3: 0.05555555555555555, 4: 0.018518518518518517, 5: 0.018518518518518517}, 'WC1': {1: 0.018518518518518517, 2: 0.018518518518518517, 3: 0.018518518518518517, 4: 0.07407407407407407, 5: 0.8703703703703703}, 'WC6': {1: 0.018518518518518517, 2: 0.14814814814814814, 3: 0.5185185185185185, 4: 0.2777777777777778, 5: 0.037037037037037035}, 'WC3': {1: 0.018518518518518517, 2: 0.018518518518518517, 3: 0.018518518518518517, 4: 0.16666666666666666, 5: 0.7777777777777778}, 'WC8': {1: 0.5555555555555556, 2: 0.35185185185185186, 3: 0.05555555555555

In [2]:
from collections import defaultdict

output = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float))))

def main():
    for context in contexts:
        #print("----------------------")
        #print("  CONTEXT: %s" % context)
        #print("----------------------")
        for u in utterances:
            #print("--- utterance: %s ----" % u)
            total_prob = 0.0
            for s in states:
                for A in affects:
                    prob = pragmatic_listener(s, A, u, context)
                    output[context][u][s][A] = prob
                    #print("  s: %s, A: %s:\t%f" % (s, A, prob))
                    total_prob += prob
            #print("  total_prob: %f" % total_prob)
main()

In [3]:
#example on marginalizing over the affect to get the probability of a state. This is done over every utterance/context combination
# for c in range(len(contexts)):
#     for u in output[contexts[c]].keys():
#         out = output[contexts[c]][u]
#         y = [sum(affect.values()) for affect in out.values()]
#         x = list(out.keys())

In [4]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10, 10))
labels = ["terrible", "bad", "neutral", "good", "amazing"]


for c in range(len(contexts)):
    for u in output[contexts[c]].keys():
        out = output[contexts[c]][u]
        y = [sum(affect.values()) for affect in out.values()]
        x = list(out.keys())

        ax1 = plt.subplot2grid((9,5),(c,u-1))
        ax1.set_ylim(0, 1)
        ax1.plot(list(out.keys()),y)
        plt.grid()
        plt.xticks(list(out.keys()), labels) 
        plt.yticks(np.arange(0.0, 1.0, 0.25))


fig.subplots_adjust(hspace=.5, wspace=.5)
plt.show()

<Figure size 1000x1000 with 45 Axes>

In [5]:
#write output 
def ddict2dict(d):
    '''
    convert recursive defaultdict to dict
    '''
    for k, v in d.items():
        if isinstance(v, dict):
            d[k] = ddict2dict(v)
    return dict(d)

# for c in range(len(contexts)):
#     for u in output[contexts[c]].keys():
#         out = output[contexts[c]][u]
#         y = [sum(affect.values()) for affect in out.values()]
#         x = list(out.keys())

for c in range(len(contexts)):
    for u in output[contexts[c]].keys():
        out = output[contexts[c]][u]
        print([sum(affect.values()) for affect in out.values()])
        #output[contexts[c]][u] = [sum(affect.values()) for affect in out.values()]
        
output = ddict2dict(output)
with open('fig_5_labels.json', 'w') as fp:
    fp.write(str(output))

[0.0774852682314899, 0.025887017529188767, 0.017642718099620248, 0.04830869249768929, 0.8306763036420118]
[0.03375715926366151, 0.09305849428153387, 0.03253254606674466, 0.08467644197778239, 0.7559753584102775]
[0.027731109449354654, 0.03921352903527142, 0.12094234770911513, 0.13907921028854653, 0.6730338035177125]
[0.007621430456218706, 0.010244480358502141, 0.013959558207616383, 0.18676564179605332, 0.7814088891816093]
[0.004255022134088909, 0.0029695713636076716, 0.002193335810130856, 0.025370974062492425, 0.9652110966296803]
[0.07705401304151718, 0.0257429396816549, 0.01754452506345554, 0.03602986774158341, 0.843628654471789]
[0.03392967566319452, 0.09353407092155923, 0.032698804064677504, 0.06383188652322867, 0.7760055628273401]
[0.028310048075708708, 0.04003218458439887, 0.12346724476664626, 0.106487061132312, 0.7017034614409341]
[0.007857676930228912, 0.010562035229161385, 0.014392271780774908, 0.14441620408372316, 0.8227718119761116]
[0.004195473004987872, 0.0029280121465381508